<a href="https://colab.research.google.com/github/arina19-2000/unimi/blob/main/tesi/tesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [726]:
import numpy as np
import pandas as pd

In [727]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/colab


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/colab


## Data preparation

In [728]:
dfa = pd.read_excel('results adults2.xlsx')

In [729]:
column_types = dfa.dtypes

type_counts = column_types.value_counts()
print("Column types and counts:")
print(type_counts)

Column types and counts:
object     89
float64    20
int64       1
Name: count, dtype: int64


In [730]:
dfa = dfa.dropna(axis=1, how='all')
dfa.shape

(786, 109)

In [731]:
dfa = dfa.dropna(thresh=int(0.05 * dfa.shape[1]), axis=0)
dfa.shape

(495, 109)

In [732]:
dfa = dfa.dropna(thresh=int(0.05 * dfa.shape[1]), axis=1)
dfa.shape

(495, 105)

In [733]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('italian'))

def remove_italian_stopwords(col):
    col = col.lower()
    col = re.sub(r'[^\w\s]', '', col)
    # col = re.sub(r'^G\d+\[.*?\]\.\s*', '', col)
    col = re.sub(r'\s+', '_', col)
    words = [word for word in col.split('_') if word not in stop_words]
    return '_'.join(words)

dfa.columns = [remove_italian_stopwords(col) for col in dfa.columns]

dfa.columns


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Index(['id_risposta', 'figli_adottati_nati_prima_2002_escluso',
       'numero_figli_adottati_nati_prima_2002_escluso_n', 'genere',
       'anno_nascita_anno_nascita', 'adozione', 'anno_arrivo_famiglia_anno',
       'stato_provenienza', 'scolarizzato_paese_origine_italia_inclusa',
       'età_inserimento_scuola_età',
       ...
       'valutazione_voto603', 'conseguito_laurea_triennale_magistrale3',
       'voto_laurea_voto110',
       'professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_no',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado',
       'padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori',
       'padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipolo

In [734]:
dfa = dfa.loc[:, dfa.apply(pd.Series.nunique) > 1]
dfa.shape

(495, 104)

In [735]:
dfa.head()

,id_risposta,numero_figli_adottati_nati_prima_2002_escluso_n,genere,anno_nascita_anno_nascita,adozione,anno_arrivo_famiglia_anno,stato_provenienza,scolarizzato_paese_origine_italia_inclusa,età_inserimento_scuola_età,prima_classe_frequenza_scuola,...,valutazione_voto603,conseguito_laurea_triennale_magistrale3,voto_laurea_voto110,professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,padre_mai_stato_organi_collegiali_scuole_propri_figli_no,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore
0,1,1.0,Femmina,1999.0,Nazionale,2004.0,Italia,No,5.0,Scuola materna,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,programmista,No,No,No,Sì,"Sì, 1 anno",Sì
2,3,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,No,6.0,Prima elementare,...,NaN,NaN,NaN,Operaio,Sì,No,No,No,NaN,NaN
3,4,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,No,6.0,Prima elementare,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,Impiegato,Sì,No,No,No,No,No
4,5,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,Sì,4.0,Scuola materna,...,NaN,NaN,110.0,Assistente sociale,No,No,No,Sì,"Sì, 1 anno",No
5,6,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,No,6.0,Scuola materna,...,60.0,NaN,NaN,Impiegato statale,Sì,No,No,No,No,No


## Rename column content

In [736]:
dfa['genere_fem'] = dfa['genere'].apply(lambda x: 1 if x == 'Femmina' else 0)
dfa['genere_fem1'] = dfa['genere1'].apply(lambda x: 1 if x == 'Femmina' else 0)
dfa['adozione_int'] = dfa['adozione'].apply(lambda x: 1 if x == 'Internazionale' else 0)
dfa['adozione_int1'] = dfa['adozione1'].apply(lambda x: 1 if x == 'Internazionale' else 0)

In [737]:
dfa['adopt_age'] = dfa['anno_arrivo_famiglia_anno'] - dfa['anno_nascita_anno_nascita']


In [738]:
for col in dfa.columns:
    if isinstance(dfa[col], pd.Series):
        unique_values = dfa[col].dropna().unique()
        if set(unique_values).issubset({'Sì', 'No'}):
            dfa[col] = dfa[col].replace({'Sì': 1, 'No': 0})
dfa.head()

,id_risposta,numero_figli_adottati_nati_prima_2002_escluso_n,genere,anno_nascita_anno_nascita,adozione,anno_arrivo_famiglia_anno,stato_provenienza,scolarizzato_paese_origine_italia_inclusa,età_inserimento_scuola_età,prima_classe_frequenza_scuola,...,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore,genere_fem,genere_fem1,adozione_int,adozione_int1,adopt_age
0,1,1.0,Femmina,1999.0,Nazionale,2004.0,Italia,0.0,5.0,Scuola materna,...,0.0,0.0,1.0,"Sì, 1 anno",1.0,1,0,0,0,5.0
2,3,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,0.0,6.0,Prima elementare,...,0.0,0.0,0.0,NaN,NaN,1,1,0,0,1.0
3,4,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,0.0,6.0,Prima elementare,...,0.0,0.0,0.0,No,0.0,1,0,1,0,7.0
4,5,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,1.0,4.0,Scuola materna,...,0.0,0.0,1.0,"Sì, 1 anno",0.0,1,0,0,0,5.0
5,6,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,0.0,6.0,Scuola materna,...,0.0,0.0,0.0,No,0.0,1,0,1,0,15.0


In [739]:
dfa = dfa.drop(columns=['genere','adozione','scuola_secondaria_secondo_grado_superiore_frequentato_altro', 'genere1','adozione1','prima_classe_frequenza_scuola_altro',])

In [740]:
hierarchical_mapping = {
    'Asilo nido': 1,
    'Scuola materna': 2,
    'Scuola d\'infanzia': 3,
    'Prima elementare': 4,
    'Seconda elementare': 5,
    'Terza elementare': 6,
    'Quarta elementare': 7,
    'Prima media': 8,
    'Seconda media': 9,
    'Terza media': 10,
    'Altro': 0
}

dfa['prima_classe_frequenza_scuola'] = dfa['prima_classe_frequenza_scuola'].map(hierarchical_mapping)
dfa['prima_classe_frequenza_scuola1'] = dfa['prima_classe_frequenza_scuola1'].map(hierarchical_mapping)


In [741]:
hierarchical_mapping2 = {
    'No': 0,
    'Sì, 1 anno': 1,
    'Sì, 2 anni': 2,
    'Sì, più di 2 anni': 3,
}

dfa['dovuto_ripetere_anni_superiore'] = dfa['dovuto_ripetere_anni_scuola_secondaria_superiore'].map(hierarchical_mapping2).fillna(-1)
dfa['dovuto_ripetere_anni_superiore1'] = dfa['dovuto_ripetere_anni_scuola_secondaria_superiore1'].map(hierarchical_mapping2).fillna(-1)
dfa['dovuto_ripetere_anni_superiore2'] = dfa['dovuto_ripetere_anni_scuola_secondaria_superiore2'].map(hierarchical_mapping2).fillna(-1)
dfa['dovuto_ripetere_anni_superiore3'] = dfa['dovuto_ripetere_anni_scuola_secondaria_superiore3'].map(hierarchical_mapping2).fillna(-1)
dfa['dovuto_ripetere_anni_madre'] = dfa['padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori'].map(hierarchical_mapping2).fillna(-1)
dfa['dovuto_ripetere_anni_padre'] = dfa['madre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori'].map(hierarchical_mapping2).fillna(-1)
dfa['dovuto_ripetere_anni_corso_formazione'] = dfa['dovuto_ripetere_anni_scuola_professionale_durante_corso_formazione_frequentato'].map(hierarchical_mapping2).fillna(-1)

dfa = dfa.drop([
    'dovuto_ripetere_anni_scuola_secondaria_superiore',
    'dovuto_ripetere_anni_scuola_secondaria_superiore1',
    'dovuto_ripetere_anni_scuola_secondaria_superiore2',
    'dovuto_ripetere_anni_scuola_secondaria_superiore3',
    'padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori',
    'madre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori',
    'dovuto_ripetere_anni_scuola_professionale_durante_corso_formazione_frequentato'
], axis=1)

In [742]:
dfa['stato_provenienza'] = dfa['stato_provenienza'].str.lower()

In [743]:
dfa['stato_provenienza1'] = dfa['stato_provenienza1'].str.lower()
dfa['stato_provenienza1'].value_counts()

,count
stato_provenienza1,
colombia,13
russia,12
brasile,9
india,8
italia,6
cambogia,5
etiopia,4
romania,4
bolivia,4


In [744]:
region_mapping = {
    # Europe
    'italia': 'Italy',
    'russia': 'East Europe',
    'ucraina': 'East Europe',
    'ungheria': 'East Europe',
    'bulgaria': 'East Europe',
    'bielorussia': 'East Europe',
    'romania': 'East Europe',
    'polonia': 'East Europe',
    'lituania': 'East Europe',
    'moldavia': 'East Europe',
    'ukraina': 'East Europe',
    'karelia': 'East Europe',

    # Latin America
    'brasile': 'Latin America',
    'colombia': 'Latin America',
    'bolivia': 'Latin America',
    'perù': 'Latin America',
    'peru': 'Latin America',
    'peru’': 'Latin America',
    'cile': 'Latin America',
    'ecuador': 'Latin America',
    'messico': 'Latin America',
    'guatemala': 'Latin America',
    'america latina': 'Latin America',

    # Asia
    'india': 'Asia',
    'cambogia': 'Asia',
    'nepal': 'Asia',
    'vietnam': 'Asia',
    'cina': 'Asia',
    'sri lanka': 'Asia',
    'filippine': 'Asia',
    'kazakistan': 'Asia',
    'thailandia': 'Asia',
    'thailand': 'Asia',
    'korea': 'Asia',
    'cambigia': 'Asia',
    'brazile': 'Asia',

    # Africa
    'etiopia': 'Africa',
    'madagascar': 'Africa',
    'marocco': 'Africa',
    'kenya': 'Africa',
    'repubblica democratica del congo': 'Africa',
    'costa d\'avorio': 'Africa',
    'burkina faso': 'Africa',
    'mauritania': 'Africa',

    # Other/Unknown
    'other': 'Other',
}

dfa['origin_region'] = dfa['stato_provenienza'].str.lower().map(region_mapping)
dfa['origin_region'].fillna('Other', inplace=True)
dfa[['stato_provenienza', 'origin_region']].head()


,stato_provenienza,origin_region
0,italia,Italy
2,italia,Italy
3,ucraina,East Europe
4,italia,Italy
5,ucraina,East Europe


In [745]:
# Define the mapping for regions based on the country names in 'stato_provenienza1'
region_mapping_1 = {
    'italia': 'Italy',
    'russia': 'East Europe',
    'brasile': 'South America',
    'colombia': 'South America',
    'ucraina': 'East Europe',
    'india': 'Asia',
    'cambogia': 'Asia',
    'etiopia': 'Africa',
    'romania': 'East Europe',
    'ungheria': 'East Europe',
    'nepal': 'Asia',
    'bolivia': 'South America',
    'perù': 'South America',
    'kazakistan': 'Asia',
    'bulgaria': 'East Europe',
    'vietnam': 'Asia',
    'polonia': 'East Europe',
    'ghana': 'Africa',
    'moldavia': 'East Europe',
    'filippine': 'Asia',
    'thailandia': 'Asia',
    'messico': 'South America',
    'costa d\'avorio': 'Africa',
    'mauritania': 'Africa',
    'cile': 'South America',
    'nazionale': 'Italy',
    'rissia': 'East Europe',
    'other': 'Other'
}

dfa['origin_region1'] = dfa['stato_provenienza1'].map(region_mapping_1).fillna('No sibling')
dfa['origin_region1'].value_counts()


,count
origin_region1,
No sibling,398
South America,33
East Europe,28
Asia,22
Italy,7
Africa,7


In [746]:
from sklearn.preprocessing import LabelEncoder

frequency_map = dfa['origin_region'].value_counts().to_dict()
dfa['origin_region'] = dfa['origin_region'].map(frequency_map)

frequency_map2 = dfa['origin_region1'].value_counts().to_dict()
dfa['origin_region1'] = dfa['origin_region1'].map(frequency_map2)

In [747]:
experience_mapping = {
    'Meno impegnativo': 1,
    'Impegnativo nello stesso modo': 2,
    'Più impegnativo': 3
}

column_rename_mapping = {
    'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_primaria': 'school_experience_primary_encoded',
    'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_primaria1': 'school_experience_primary1_encoded',
    'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_secondaria_grado1': 'school_experience_secondary_encoded',
    'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_secondaria_ii_grado1': 'school_experience_secondary2_encoded',
    'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_università1': 'school_experience_university1_encoded',
    'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_secondaria_grado': 'school_experience_secondary_encoded',
    'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_secondaria_ii_grado': 'school_experience_secondary2_encoded',
    'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_università': 'school_experience_university_encoded'
}

for old_column, new_column in column_rename_mapping.items():
    dfa[new_column] = dfa[old_column].map(experience_mapping)

dfa.drop(columns=column_rename_mapping.keys(), inplace=True)

In [748]:
region_classification = {
    'Lombardia': 'Northern',
    'Emilia Romagna': 'Northern',
    'Veneto': 'Northern',
    'Piemonte': 'Northern',
    'Liguria': 'Northern',
    'Friuli-Venezia Giulia': 'Northern',
    'Valle d\'Aosta': 'Northern',
    'P.A. Trento': 'Northern',
    'P.A. Bolzano': 'Northern',

    'Toscana': 'Central',
    'Lazio': 'Central',
    'Marche': 'Central',
    'Umbria': 'Central',
    'Abruzzo': 'Central',

    'Puglia': 'Southern',
    'Campania': 'Southern',
    'Sicilia': 'Southern',
    'Sardegna': 'Southern',
    'Calabria': 'Southern',
    'Molise': 'Southern',
    'Basilicata': 'Southern'
}

dfa['region_area_madre'] = dfa['regione_residenza_madre'].map(region_classification)
dfa['region_area_padre'] = dfa['regione_residenza_padre'].map(region_classification)

mother_freq_encoding = dfa['region_area_madre'].value_counts(normalize=False)
dfa['region_area_madre_freq_encoded'] = dfa['region_area_madre'].map(mother_freq_encoding)

father_freq_encoding = dfa['region_area_padre'].value_counts(normalize=False)
dfa['region_area_padre_freq_encoded'] = dfa['region_area_padre'].map(father_freq_encoding)

1. Nessun titolo/Licenza elementare o frequentazione della sola scuola primaria
2. Licenza di scuola secondaria di primo grado (precedentemente denominata licenza media inferiore)
3. Titolo di diploma di qualifica professionale che non permette l’accesso all’università (3 o 4 anni)
4. Titolo di scuola secondaria di secondo grado (denominata precedentemente scuola superiore) che permette l’iscrizione all’università (5 anni)
5. Titolo post-secondario non terziario (IFTS)
6. Titolo di studio universitario
7. Titolo di studio post-universitario (Master, Scuola di perfezionamento, ecc.)
8. Dottorato di ricerca

In [749]:
dfa['scuola_professionale_corso_formazione_professionale_frequentato'] = dfa['scuola_professionale_corso_formazione_professionale_frequentato'].notna().astype(int)


In [750]:
maps = {
    '1. Liceo': 1,
    '2. Istituto tecnico': 2,
    '3. Istituto professionale': 3,
    'Altro': 4
}

dfa['scuola_secondaria_secondo_grado_superiore_frequentato'] = dfa['scuola_secondaria_secondo_grado_superiore_frequentato'].map(maps).fillna(0)
dfa['scuola_secondaria_secondo_grado_superiore_frequentato1'] = dfa['scuola_secondaria_secondo_grado_superiore_frequentato1'].map(maps).fillna(0)

In [751]:
mapping = {
    'Nessun titolo/Licenza elementare o frequentazione della sola scuola primaria': 0.5,
    'Licenza di scuola secondaria di primo grado (precedentemente denominata licenza media inferiore)': 1,
    'Titolo di diploma di qualifica professionale che non permette l’accesso all’università (3 o 4 anni)': 2,
    'Titolo di scuola secondaria di secondo grado (denominata precedentemente scuola superiore) che permette l’iscrizione all’università (5 anni)': 3,
    'Titolo post-secondario non terziario (IFTS)': 4,
    'Titolo di studio universitario': 5,
    'Titolo di studio post-universitario (Master, Scuola di perfezionamento, ecc.)': 6,
    'Dottorato di ricerca': 7,
}

for col in ['qual_titolo_studio_massimo_conseguito','qual_titolo_studio_massimo_conseguito1', 'qual_titolo_studio_massimo_conseguito_madre', 'qual_titolo_studio_massimo_conseguito_padre']:
    dfa[col] = dfa[col].str.extract(r'(\d)\.\s*(.*)')[1].map(mapping)
    dfa[col].fillna(0, inplace=True)

In [752]:
education_mapping = {
    'Scuola secondaria di primo grado': 2,
    'Istituto di formazione professionale che non permette l’accesso all’università 6 Istituto post-secondario non terziario (IFTS)': 3,
    'Scuola secondaria di secondo grado che permette l’iscrizione all’università (5 anni)': 4,
    'Titolo post-secondario non terziario (IFTS)': 5,
    'Università': 6,
    'Master, Scuola di perfezionamento, ecc.': 7
}

dfa['attualmente_frequentando_enc'] = dfa['attualmente_frequentando'].map(education_mapping).fillna(0)
dfa['attualmente_frequentando_enc1'] = dfa['attualmente_frequentando1'].map(education_mapping).fillna(0)
dfa = dfa.drop(columns=['attualmente_frequentando','attualmente_frequentando1'])

In [753]:
hierarchical_mapping3 = {
    'Laurea triennale': 1,
    'Laurea magistrale': 2,
}
dfa['conseguito_laurea_triennale_magistrale'] = dfa['conseguito_laurea_triennale_magistrale'].map(hierarchical_mapping3).fillna(0)
dfa['conseguito_laurea_triennale_magistrale1'] = dfa['conseguito_laurea_triennale_magistrale1'].map(hierarchical_mapping3).fillna(0)
dfa['conseguito_laurea_triennale_magistrale2'] = dfa['conseguito_laurea_triennale_magistrale2'].map(hierarchical_mapping3).fillna(0)
dfa['conseguito_laurea_triennale_magistrale3'] = dfa['conseguito_laurea_triennale_magistrale3'].map(hierarchical_mapping3).fillna(0)

In [754]:
frequency_encoding = dfa['descrizione_nucleo_familiare_figlio'].value_counts()

dfa['descrizione_nucleo_familiare_figlio_encoded'] = dfa['descrizione_nucleo_familiare_figlio'].map(frequency_encoding)
dfa['descrizione_nucleo_familiare_figlio_encoded1'] = dfa['descrizione_nucleo_familiare_figlio1'].map(frequency_encoding)
dfa = dfa.drop(columns=['descrizione_nucleo_familiare_figlio','descrizione_nucleo_familiare_figlio1'])

In [755]:
dfa['fatto_esperienze_allestero'] = dfa['fatto_esperienze_allestero'].replace('No', 0).fillna(0)
dfa['fatto_esperienze_allestero'] = dfa['fatto_esperienze_allestero'].apply(lambda x: 1 if x is not None and x != 0 else x)
dfa['fatto_esperienze_allestero'] = dfa['fatto_esperienze_allestero'].astype(int)

dfa['fatto_esperienze_allestero1'] = dfa['fatto_esperienze_allestero1'].replace('No', 0).fillna(0)
dfa['fatto_esperienze_allestero1'] = dfa['fatto_esperienze_allestero1'].apply(lambda x: 1 if x is not None and x != 0 else x)
dfa['fatto_esperienze_allestero1'] = dfa['fatto_esperienze_allestero1'].astype(int)

In [756]:
dfa['left_school_cause_difficult'] = dfa['lasciato'].fillna(0).where(dfa['lasciato'].isna(), 1)
dfa['left_school_cause_difficult1'] = dfa['lasciato1'].fillna(0).where(dfa['lasciato1'].isna(), 1)
dfa = dfa.drop(columns=['lasciato', 'lasciato1'])


In [757]:
non_numeric_columns = dfa.select_dtypes(exclude=[np.number]).columns
print(non_numeric_columns)

Index(['stato_provenienza',
       'ambito_titolo_universitario_es_economia_giurispudenza_medicina_etc',
       'lavoro_fa', 'stato_provenienza1', 'tipologia_cambio1',
       'ambito_titolo_universitario_es_economia_giurispudenza_medicina_etc',
       'lavoro_fa1', 'descrizione_nucleo_familiare_figlio_altro',
       'regione_residenza_madre',
       'professione_madre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore',
       'regione_residenza_padre',
       'professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore',
       'region_area_madre', 'region_area_padre', 'left_school_cause_difficult',
       'left_school_cause_difficult1'],
      dtype='object')


In [758]:
dfa = dfa.drop(columns=[
    'stato_provenienza',
    'ambito_titolo_universitario_es_economia_giurispudenza_medicina_etc',
    'lavoro_fa',
    'stato_provenienza1',
    'tipologia_cambio1',
    'ambito_titolo_universitario_es_economia_giurispudenza_medicina_etc',
    'lavoro_fa1',
    'descrizione_nucleo_familiare_figlio_altro',
    'regione_residenza_madre',
    'professione_madre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore',
    'regione_residenza_padre',
    'professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore',
    'region_area_madre',
    'region_area_padre'
])

In [759]:
dfa.to_excel('draft.xlsx', index=False)


In [760]:
# dfa = dfa.drop(columns=['genere','adozio])

In [761]:
# le = LabelEncoder()
# dfa[''] = le.fit_transform(dfa[''])

In [762]:
# categorical_cols = dfa.select_dtypes(include=['object']).columns

# for col in categorical_cols:
#     if col in dfa.columns:
#         dfa[col] = dfa[col].fillna('No response')
#     else:
#         print(f"Column {col} does not exist in the DataFrame.")

##Children survey results

In [763]:
dfc = pd.read_excel('results kids.xlsx')

In [764]:
dfc.shape

(235, 93)

In [765]:
dfc = dfc.dropna(axis=1, how='all')
dfc.shape

(235, 91)

In [766]:
dfc.columns = [remove_italian_stopwords(col) for col in dfc.columns]

dfc.columns

Index(['id_id_risposta', 'submitdate_data_invio', 'lastpage_ultima_pagina',
       'startlanguage_lingua_iniziale', 'seed_seme', 'startdate_data_inizio',
       'datestamp_data_dellultima_azione', 'refurl_url_riferimento',
       'privacysq001_base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_leg

In [767]:
column_types = dfc.dtypes
type_counts = column_types.value_counts()

print("Column types and counts:")
print(type_counts)

Column types and counts:
object            72
float64           14
datetime64[ns]     3
int64              2
Name: count, dtype: int64


In [768]:
categorical_cols = dfc.select_dtypes(include=['object']).columns

for col in categorical_cols:
    if col in dfc.columns:
        dfc[col] = dfc[col].fillna('No response')
    else:
        print(f"Column {col} does not exist in the DataFrame.")

<ipython-input-768-d1745a59488e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc[col] = dfc[col].fillna('No response')


In [769]:
dfc.head()

,id_id_risposta,submitdate_data_invio,lastpage_ultima_pagina,startlanguage_lingua_iniziale,seed_seme,startdate_data_inizio,datestamp_data_dellultima_azione,refurl_url_riferimento,privacysq001_base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_legale_via_liberiana_n17_roma_infocoordinamentocareorg_persona_legale_rappresentante_preghiamo_quindi_manifestare_consenso_scritto_trattamento_parte_dati_classificati_sensibili_cliccando_caselle_qui_sotto_doppia_autorizzazione_requisito_necessario_proseguimento_questionario_letto_obbligatoria,privacysq002_base_previsto_regolamento_ue_n_679_2016_d_lgs_30_giugno_2003_n_196_smi_nonché_dallautorizzazione_garante_privacy_n_52002_trattamento_dati_parte_diverse_categorie_titolari_dati_riguardano_vengono_liberamente_comunicati_registrati_questionario_online_protetti_anonimi_questionario_infatti_chiede_dati_identificativi_traccia_lip_memorizza_cookies_rispondenti_nessun_modo_possibile_risalire_compilatore_risposte_questionario_trattate_via_riservata_forma_aggregata_coordinamento_care_tramite_collaboratori_specificatamente_incaricati_finalità_connesse_propria_attività_ricerca_comunicati_terzi_forma_aggregata_anonima_rispondere_parte_volontario_facoltativo_eventuale_rifiuto_interruzione_alcuna_conseguenza_titolare_trattamento_dati_coordinamento_care_sede_legale_via_liberiana_n17_roma_infocoordinamentocareorg_persona_legale_rappresentante_preghiamo_quindi_manifestare_consenso_scritto_trattamento_parte_dati_classificati_sensibili_cliccando_caselle_qui_sotto_doppia_autorizzazione_requisito_necessario_proseguimento_questionario_manifesto_consenso_obbligatoria,...,g8q00003a_conseguito_laurea_triennale_magistrale,g8q00004sq001_sq001_voto_laurea_voto110,g8q00005_professione_padre_quando_fatto_scelta_scuola_secondaria_ii_grado_superiore,g8q00006sq001_padre_mai_stato_organi_collegiali_scuole_propri_figli_no,g8q00006sq002_padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,g8q00006sq005_padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,g8q00006sq004_padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,g8q00007_padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,g8q00008_padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore,g8q00009_tipologia_cambio
0,1,NaT,0.0,it,433164667,2021-07-03 08:57:45,2021-07-03 08:57:51,No response,No,No,...,No response,NaN,No response,No response,No response,No response,No response,No response,No response,No response
1,2,2021-07-06 16:52:01,5.0,it,696014843,2021-07-06 16:48:47,2021-07-06 16:52:01,http://m.facebook.com/,Sì,Sì,...,No response,NaN,Impiegato,Sì,No,No,No,No response,No response,No response
2,3,2021-07-06 16:55:44,5.0,it,531131002,2021-07-06 16:49:03,2021-07-06 16:55:44,No response,Sì,Sì,...,No response,110.0,Docente universitario,Sì,No,No,No,No,No,No response
3,4,NaT,0.0,it,1471963680,2021-07-06 16:49:26,2021-07-06 16:49:39,http://m.facebook.com/,No,No,...,No response,NaN,No response,No response,No response,No response,No response,No response,No response,No response
4,5,NaT,2.0,it,1652562375,2021-07-06 16:54:19,2021-07-06 16:54:47,https://m.facebook.com/,Sì,Sì,...,No response,NaN,No response,No response,No response,No response,No 